In [ ]:
import ftplib 
from ftplib import FTP

In [ ]:
HOSTNAME = 'ftp.datasus.gov.br'
ftp_server = ftplib.FTP(HOSTNAME)
ftp_server.login() 
ftp_server.cwd('dissemin/publicos/SINAN/DADOS/FINAIS/')

In [ ]:
filename = "LEIVBR07.dbc"
with open(filename, "wb") as file:    
    ftp_server.retrbinary(f"RETR {filename}", file.write) 